In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

# Function to determine season based on date
def get_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'

# Function to determine day of the week based on date
def get_day_of_week(date):
    return date.strftime('%A')  # Returns full weekday name (e.g., Monday)

# Function to determine if the date is a holiday
def is_holiday(date):
    # Assuming some fixed holidays (you can expand this list)
    holidays = [
        '2025-01-01', '2025-12-25', '2025-11-15'  # New Year, Christmas, Diwali (for example)
    ]
    return 1 if date.strftime('%Y-%m-%d') in holidays else 0

# Function to determine weather condition based on temperature
def get_weather_condition(temperature):
    if temperature > 30:
        return 'Sunny'
    elif 20 <= temperature <= 30:
        return 'Cloudy'
    else:
        return 'Rainy'

# Load your data
df = pd.read_csv('patient_inflow.csv')

# Encode Categorical Variables
label_encoders = {}
for col in ['Day_of_Week', 'Season', 'Weather_Condition']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])  # Convert to numbers
    label_encoders[col] = le  # Save encoder for future use

# Features (X) and Target (y)
X = df[['Day_of_Week', 'Is_Holiday', 'Season', 'Temperature', 'Weather_Condition']]  
y = df[['Flu_Cases', 'Cold_Fever_Cases', 'Respiratory_Issues', 'Infectious_Diseases', 'Accidents_Emergencies']]  

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Multi-Output Regression Model
model = MultiOutputRegressor(GradientBoostingRegressor())
model.fit(X_train, y_train)

# Input Section: Enter values manually
print("Enter the details for prediction:")

# Input date and temperature
input_date = input("Enter the Date (YYYY-MM-DD): ")
temperature = float(input("Enter the Temperature (in Celsius): "))

# Convert the input date to datetime object
date_obj = datetime.strptime(input_date, '%Y-%m-%d')

# Calculate Season, Day of Week, and Holiday
season = get_season(date_obj)
day_of_week = get_day_of_week(date_obj)
holiday = is_holiday(date_obj)

# Calculate weather condition based on temperature
weather_condition = get_weather_condition(temperature)

# Prepare input data
input_data = pd.DataFrame({
    'Day_of_Week': [day_of_week],
    'Is_Holiday': [holiday],
    'Season': [season],
    'Temperature': [temperature],
    'Weather_Condition': [weather_condition]
})

# Encode input data using the same encoders
for col in ['Day_of_Week', 'Season', 'Weather_Condition']:
    input_data[col] = label_encoders[col].transform(input_data[col])

# Predict
predicted_values = model.predict(input_data)
print("\nPredicted Patient Count by Department (Rounded): ")
total_patients = 0
for idx, dept in enumerate(['Flu Cases', 'Cold/Fever Cases', 'Respiratory Issues', 'Infectious Diseases', 'Accidents/Emergencies']):
    predicted_value = round(predicted_values[0][idx])  # Round to the nearest integer
    print(f"{dept}: {predicted_value}")
    total_patients += predicted_value

# Display total number of patients
print(f"\nTotal Number of Patients (Rounded): {total_patients}")


ModuleNotFoundError: No module named 'pandas'

In [3]:
from sklearn.metrics import mean_squared_error, r2_score

# Predict on test data
y_pred = model.predict(X_test)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Calculate R^2 score
r2 = r2_score(y_test, y_pred)
print(f"R^2 Score: {r2}")


Mean Squared Error: 85.67801018172939
R^2 Score: 0.26925729276796184
